In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,num_labels = 2)

def tokenize_function(example):
     return tokenizer(example["sentence1"],example["sentence2"],truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-train")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
from transformers import Trainer

trainer = Trainer(
     model,
     training_args,
     train_dataset=tokenized_datasets["train"],
     eval_dataset=tokenized_datasets["validation"],
     data_collator=data_collator,
     tokenizer = tokenizer

)
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape,predictions.label_ids.shape)

In [ ]:
import numpy as np
preds = np.argmax(predictions.predictions,axis=-1)
print(preds)

In [ ]:
import evaluate
metric = evaluate.load("glue", "merpc")
metric.compute(predictions=preds,references=predictions.label_ids)


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate

In [4]:
def compute_metrics(eval_preds):
     metric = eval.load("glue","mrpc")
     logits,labels = eval_preds
     predictions = np.argmax(logits,axis= -1)
     return metric.compute(predictions = predictions,references = labels)


In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)